In [1]:
%load_ext autoreload
%autoreload 2

from dataset_prepare import *
from utils import custom_RFC
import pickle

from matplotlib import pyplot as plt

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets.standard_dataset import StandardDataset
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.metrics import ClassificationMetric
from sklearn.metrics import classification_report
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from IPython.display import Markdown, display

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from tqdm import tqdm
import imblearn

from dataset_prepare import prepare_standard_dataset

2024-04-03 14:13:42.854225: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 14:13:42.910945: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 14:13:42.910989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 14:13:42.912271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 14:13:42.924095: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df = load_dataset()
df

,Num_Acc,trajet,catr,circ,nbv,prof,plan,surf,infra,situ,...,atm,col,catv,obs,obsm,choc,mortal,pieton,sexe_conducteur,age
0,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,14
1,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,74
2,202200000002,0,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,34
3,202200000002,4,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,52
4,202200000003,0,3,-1,2,1,1,1,5,1,...,1,2,3,0,2,1,0,0,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126656,202200055300,5,3,2,2,1,2,7,0,3,...,9,6,3,1,0,1,1,0,1,27
126657,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,20
126659,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,69
126660,202200055302,1,3,3,4,1,1,1,0,1,...,1,2,5,0,2,1,0,0,1,30


In [3]:
label = 'mortal'

# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df)
df = df.drop(columns='Num_Acc')

In [4]:
dataset_orig_train, dataset_orig_test = prepare_standard_dataset(X_train, y_train, X_test, y_test, label)

In [5]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(117678, 24)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sexe_conducteur']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['trajet', 'catr', 'circ', 'nbv', 'prof', 'plan', 'surf', 'infra', 'situ', 'vma', 'mois', 'lum', 'dep', 'agg', 'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton', 'sexe_conducteur', 'age']


In [6]:
privileged_groups=[{'sexe_conducteur': 1}]
unprivileged_groups = [{'sexe_conducteur' : 0}]

## Repondération
Utilisation de l'outil de repondération de `aif360` et affichage du résultat obtenu.

In [14]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.080457


In [15]:
# 0 -> Femme et 1 -> Homme
RW = Reweighing(privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

In [16]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [17]:
# valeurs catégorielles
categorical_features = ['trajet', 'catr', 'circ', 'nbv', 'prof',
                        'plan', 'surf', 'vma', 'lum', 'agg', 
                        'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton',
                        'sexe_conducteur', 'infra', 'situ']
# valeurs numériques
numerical_features = ['dep','age', 'mois']

print("numerical : ", numerical_features)
print("categorical : ", categorical_features)

numerical :  ['dep', 'age', 'mois']
categorical :  ['trajet', 'catr', 'circ', 'nbv', 'prof', 'plan', 'surf', 'vma', 'lum', 'agg', 'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton', 'sexe_conducteur', 'infra', 'situ']


In [18]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [20]:
X_train = dataset_transf_train.convert_to_dataframe()[0]
y_train = dataset_transf_train.convert_to_dataframe()[0]['mortal']

In [21]:
from utils import custom_RFC
import imblearn

dt = custom_RFC(random_state=42)
clf = imblearn.pipeline.Pipeline(
    [
        ('preprocessor', transformations),
        ("resample", imblearn.over_sampling.SMOTE(random_state=42)),
        ('classifier', dt)
    ])
clf = clf.fit(X_train, y_train, classifier__sample_weight=dataset_orig_train.instance_weights)

preds = clf.predict(X_test)

clf.score(X_train, y_train), clf.score(X_test, y_test)

(0.9891398562178148, 0.7555738243998684)

In [22]:
pickle.dump(clf, open('models/rfc_reweight_model.sav', 'wb'))

In [184]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
tn, fp, fn, tp

(21968, 6749, 684, 1009)

In [185]:
import plotly.express as px

fig = px.imshow([[tn, fp], [fn, tp]], text_auto=True, labels=dict(y="Truth", x="Pred"),
                x=["False", "True"],
                y=["False", "True"]
               )
fig.show()

## Disparate impact remover
Utilisation de l'outil de `Disparate impact remover` de `aif360` et affichage du résultat obtenu.

In [53]:
label = 'mortal'

df = load_dataset()
# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df)

In [54]:
dataset_orig_train, dataset_orig_valid, dataset_orig_test = prepare_standard_dataset(X_train, y_train, X_test, y_test, label)

In [57]:
protected = 'sexe_conducteur'

index = dataset_orig_train.feature_names.index(protected)

DIs = []
for level in tqdm(np.linspace(0., 1., 11)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(dataset_orig_train)
    test_repd = di.fit_transform(dataset_orig_test)

    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    
    y_tr = train_repd.labels.ravel()
    
    dt = custom_RFC(random_state=42)
    dt.fit(X_tr, y_tr)
    
    test_repd_pred = test_repd.copy()
    test_repd_pred.labels = dt.predict(X_te)

    p = [{protected: 1}]
    u = [{protected: 0}]
    cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=p, unprivileged_groups=u)
    DIs.append(cm.disparate_impact())

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [05:50<00:00, 31.82s/it]


In [64]:
fig = plt.figure()
plt.plot(np.linspace(0, 1, 11), DIs, marker='o')
plt.plot([0, 1], [1, 1], 'g')
plt.plot([0, 1], [0.8, 0.8], 'r')
plt.ylim([0.4, 1.2])
plt.ylabel('Disparate Impact (DI)')
plt.xlabel('repair level')
fig.savefig("disparate_impact_remover.pdf")
plt.close()

plt.show()

<IPython.core.display.Javascript object>

### Apprentissage de `représentation latente fair`

In [ ]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
TR = TR.fit(dataset_orig_train, maxiter=5000, maxfun=5000)

In [11]:
# Transform training data and align features
dataset_transf_train = TR.transform(dataset_orig_train)
dataset_transf_test = TR.transform(dataset_orig_test)

In [17]:
print(classification_report(dataset_orig_test.labels, dataset_transf_test.labels))

              precision    recall  f1-score   support

         0.0       0.91      0.05      0.10     14344
         1.0       0.05      0.91      0.10       861

    accuracy                           0.10     15205
   macro avg       0.48      0.48      0.10     15205
weighted avg       0.86      0.10      0.10     15205



In [18]:
dataset_transf_test.features

array([[ 2.67691473,  3.53385236,  2.00460161, ...,  0.19262767,
         0.48424767, 49.14376419],
       [ 2.67691473,  3.53385236,  2.00460161, ...,  0.19262767,
         0.48424767, 49.14376419],
       [ 2.67691473,  3.53385236,  2.00460161, ...,  0.19262767,
         0.48424767, 49.14376419],
       ...,
       [ 2.67691473,  3.53385236,  2.00460161, ...,  0.19262767,
         0.48424767, 49.14376419],
       [ 2.67691473,  3.53385236,  2.00460161, ...,  0.19262767,
         0.48424767, 49.14376419],
       [ 2.67691473,  3.53385236,  2.00460161, ...,  0.19262767,
         0.48424767, 49.14376419]])

In [20]:
dataset_orig_test.features

array([[ 5.,  2.,  1., ...,  0.,  1., 68.],
       [ 0.,  3.,  2., ...,  0.,  1., 13.],
       [ 4.,  4.,  2., ...,  0.,  1., 44.],
       ...,
       [ 1.,  2.,  3., ...,  0.,  1., 24.],
       [ 2.,  5.,  2., ...,  0.,  1., 15.],
       [ 0.,  4.,  1., ...,  0.,  1., 32.]])

### In processing `Adversarial debiasing`

In [128]:
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [129]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)
plain_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 5.225278
epoch 0; iter: 200; batch classifier loss: 1.247293
epoch 0; iter: 400; batch classifier loss: 0.910505
epoch 0; iter: 600; batch classifier loss: 0.690384
epoch 0; iter: 800; batch classifier loss: 0.621849
epoch 1; iter: 0; batch classifier loss: 0.670442
epoch 1; iter: 200; batch classifier loss: 0.515260
epoch 1; iter: 400; batch classifier loss: 0.548557
epoch 1; iter: 600; batch classifier loss: 0.605547
epoch 1; iter: 800; batch classifier loss: 0.475843
epoch 2; iter: 0; batch classifier loss: 0.405236
epoch 2; iter: 200; batch classifier loss: 0.373213
epoch 2; iter: 400; batch classifier loss: 0.454045
epoch 2; iter: 600; batch classifier loss: 0.429714
epoch 2; iter: 800; batch classifier loss: 0.484908
epoch 3; iter: 0; batch classifier loss: 0.392919
epoch 3; iter: 200; batch classifier loss: 0.375629
epoch 3; iter: 400; batch classifier loss: 0.386292
epoch 3; iter: 600; batch classifier loss: 0.389202
epoch 3; iter: 800; 

In [106]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [107]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.082386
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.011212


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.822887
Test set: Balanced classification accuracy = 0.629412
Test set: Disparate impact = 1.068230
Test set: Equal opportunity difference = 0.051500
Test set: Average odds difference = 0.033495
Test set: Theil_index = 0.074225


In [110]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [111]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 6.177888; batch adversarial loss: 0.700714
epoch 0; iter: 200; batch classifier loss: 1.124693; batch adversarial loss: 0.712609
epoch 0; iter: 400; batch classifier loss: 0.721309; batch adversarial loss: 0.649016
epoch 0; iter: 600; batch classifier loss: 0.598603; batch adversarial loss: 0.618723
epoch 0; iter: 800; batch classifier loss: 0.647720; batch adversarial loss: 0.638899
epoch 1; iter: 0; batch classifier loss: 0.525003; batch adversarial loss: 0.640142
epoch 1; iter: 200; batch classifier loss: 0.554657; batch adversarial loss: 0.661752
epoch 1; iter: 400; batch classifier loss: 0.574018; batch adversarial loss: 0.634854
epoch 1; iter: 600; batch classifier loss: 0.550739; batch adversarial loss: 0.576887
epoch 1; iter: 800; batch classifier loss: 0.515810; batch adversarial loss: 0.628874
epoch 2; iter: 0; batch classifier loss: 0.562558; batch adversarial loss: 0.607326
epoch 2; iter: 200; batch classifier loss: 0.464451; batch a

In [112]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [113]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.082386
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.011212


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.082957
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.010330


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.822887
Test set: Balanced classification accuracy = 0.629412
Test set: Disparate impact = 1.068230
Test set: Equal opportunity difference = 0.051500
Test set: Average odds difference = 0.033495
Test set: Theil_index = 0.074225


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.811279
Test set: Balanced classification accuracy = 0.632437
Test set: Disparate impact = 1.057924
Test set: Equal opportunity difference = 0.074621
Test set: Average odds difference = 0.044141
Test set: Theil_index = 0.075338
